In [1]:
import sys

In [2]:
sys.path.append('../')

In [8]:
import gspread
import pandas
from oauth2client.service_account import ServiceAccountCredentials

In [9]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    '/Users/dshurick/Downloads/dshurick-rdb-test-cb506c785735.json', scope)

spreadsheet = gspread.authorize(credentials)

worksheet = spreadsheet.open("NFL 2018 Expected Wins")

In [5]:
# scheduled_games = pandas.DataFrame(
#     worksheet.get_worksheet(0).get_all_records())[[
#         'week',
#         'away_team',
#         'home_team',
#         'home_adv',
#         'score_away',
#         'score_home',
#         'vegas_spread',
#     ]]

In [30]:
sagarin_ratings = pandas.DataFrame(
    worksheet.get_worksheet(1).get_all_records())

scorex_ratings = pandas.DataFrame(
    worksheet.get_worksheet(2).get_all_records())

FiveThirtyEight_ratings = pandas.DataFrame(
    worksheet.get_worksheet(3).get_all_records())

massey_ratings = pandas.DataFrame(
    worksheet.get_worksheet(4).get_all_records())

# teaminfo = pandas.DataFrame(
#     worksheet.get_worksheet(6).get_all_records())

In [32]:
FiveThirtyEight_ratings['rating'] = FiveThirtyEight_ratings.rating.str.replace(
    ',', '').astype(numpy.int)

In [3]:
from src.nfl.nfl import Team, Game, Season

In [4]:
season_2018 = Season(year=2018)

In [5]:
season_2018.load_gspread()

In [1]:
import pandas
import sklearn

from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, log_loss

In [168]:
from sklearn.model_selection import GroupKFold

In [148]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

In [8]:
dtf = pandas.read_csv('./../data/processed/nfl_data.csv')
dtf = dtf.loc[(dtf.home_win == 0) | (dtf.home_win == 1)]

In [239]:
dtf

,season,five38_home_rating,five38_away_rating,massey_home_rating,massey_away_rating,sagarin_home_rating,sagarin_away_rating,home_win,gamenum,home_adv
0,2017,1434,1559,24.83,22.80,17.12,20.50,1.0,1,1
1,2017,1529,1600,21.28,18.56,23.01,21.80,1.0,2,1
2,2017,1476,1497,21.00,24.19,20.23,18.40,1.0,3,1
3,2017,1498,1480,21.69,11.05,24.08,17.84,1.0,4,1
4,2017,1623,1489,32.83,15.50,27.78,19.01,1.0,5,1
5,2017,1489,1387,29.52,23.75,24.02,11.43,0.0,6,1
6,2017,1455,1514,17.59,25.61,19.77,21.57,0.0,7,1
7,2017,1598,1436,31.03,18.36,27.66,19.74,1.0,8,1
8,2017,1282,1460,20.29,22.22,12.33,14.98,0.0,9,1
9,2017,1608,1477,24.66,21.94,26.23,20.71,0.0,10,1


In [144]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             (['five38_home_rating'], StandardScaler()),
             (['five38_away_rating'], StandardScaler()),
             (['massey_home_rating'], StandardScaler()),
             (['massey_away_rating'], StandardScaler()),
             (['sagarin_home_rating'], StandardScaler()),
             (['sagarin_away_rating'], StandardScaler()),
         ],
         sparse=True)),
    ('lm',
     SGDClassifier(
         alpha=0.08,
         loss='log',
         fit_intercept=False,
         penalty='l2',
         max_iter=1000,
         tol=1e-3,
     )),
])

In [230]:
pipe = sklearn.pipeline.Pipeline([
    ('featurize',
     DataFrameMapper(
         [
             ([
                 'five38_away_rating',
                 'five38_home_rating',
                 'massey_away_rating',
                 'massey_home_rating',
                 'sagarin_away_rating',
                 'sagarin_home_rating',
             ], StandardScaler()),
         ],
         sparse=True)),
    ('lm', LogisticRegression(C=0.4)),
])

In [232]:
x = pipe.named_steps['featurize']

In [236]:
x.features[0][1].scale_

array([97.86976773, 97.86976773,  4.51784368,  4.51784368,  4.5378788 ,
        4.5378788 ])

In [237]:
x.features[0][1].mean_

array([1513.71610169, 1513.71610169,   23.00144068,   23.00144068,
         21.25025424,   21.25025424])

In [ ]:
cv = GroupKFold(n_splits=20)

In [206]:
xxx = cross_val_score(
    pipe,
    X=dtf,
    y=dtf.home_win,
    groups=dtf.gamenum,
    scoring=make_scorer(
        log_loss,
        greater_is_better=False,
        needs_proba=True,
    ),
    cv=cv.split(X=dtf, groups=dtf.gamenum),
    n_jobs=-1,
)

In [207]:
xxx.mean()

-0.5844360835663382

In [ ]:
clf = GridSearchCV(svc, parameters, scor)

In [231]:
pipe.fit(X=dtf, y=dtf.home_win)

Pipeline(memory=None,
     steps=[('featurize', DataFrameMapper(default=False, df_out=False,
        features=[(['five38_away_rating', 'five38_home_rating', 'massey_away_rating', 'massey_home_rating', 'sagarin_away_rating', 'sagarin_home_rating'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=Fa...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [220]:
lm = pipe.named_steps['lm']

In [235]:
lm.coef_

array([[-0.48811602,  0.48811602,  0.83260993, -0.83260993,  0.17701139,
        -0.17701139]])

In [238]:
0.48811602/97.86976773

0.004987403478330498